In [17]:
# importing the libraries.
import time
import psutil
from datetime import datetime, timedelta
from typing import Any, List, Union
import sqlite3

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent
from langchain.agents.agent import AgentExecutor
from langchain_core.agents import AgentActionMessageLog
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [18]:
# checker function if ollama is running fine or not.
def check_ollama(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_ollama("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running.")

print(f"Ollama is running fine: {ollama_running}")

Ollama is running fine: True


In [19]:
# defining the LLM which will be used for our use-case.
def create_llm():
  tool_llm = ChatOllama(
          model="llama3-groq-tool-use",
          temperature=0,
  )
  return tool_llm

In [20]:
# dummy function to check if payment was successful
def payment_done():
    return True

def get_available_room(room_type):
    room_type = room_type.lower()
    conn = sqlite3.connect("hotel.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM rooms WHERE room_type = ? AND availability = 1 LIMIT 1", (room_type,))
    room = cursor.fetchone()
    room_no = room[0]
    cursor.execute("UPDATE rooms SET availability = 0 WHERE room_no = ?", (room_no,))
    conn.commit()
    return room_no
    
# function which will be called if tool calling happens.
@tool
def book_room(check_in_date, check_out_date, room_type)-> str:
    """Book a hotel's room on behalf of the user.
       Args:
           check_in_date (str): Check in date.
           check_out_date (str): Check out date.
           room_type (str): The type of room in which user wants to stay. 
       Returns:
           str: A message indicating whether the room was successfully booked or not.
    """
    room_type = room_type.lower()
    if(payment_done()):
        # update the DB that a room is booked and returns the room number.
        room_no = get_available_room(room_type)
        return f"Your {room_type} room with room number {room_no} has been booked with check in date: {check_in_date} and check out date: {check_out_date}."
    return "Payment was not successful. Room wasn't booked."

In [21]:
def integration_with_tool():
    tool_llm = create_llm()
    tools = [book_room]
    return tool_llm, tools

In [28]:
def system_prompt():
    current_date = datetime.now().strftime("%Y-%m-%d")
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""You are an AI assistant for The Lalit Hotel. There are three types of rooms available there:
        - classic (for a single person)
        - deluxe (for 2 people) 
        - mega (for 3+ people)
        
        Your primary functions are:
        1. Book rooms using the available booking tool
        2. Answer general questions about the hotel
        
        When booking rooms:
        **IMPORTANT: You MUST NEVER call the booking tool unless ALL THREE of the following pieces of information have been EXPLICITLY provided by the user:
        1. Room type (must be one of: classic, deluxe, or mega)
        2. Check-in date (must be a specific date)
        3. Check-out date (must be a specific date)
        
        If ANY of these three required pieces of information are missing, you MUST ask the user to provide the missing information before proceeding. Do not make assumptions about any of these three required fields.**
        
        Once all required information is provided:
        - Use the "book_room" function to check availability and make reservations
        - Confirm all details with the user including dates, room type, and number of guests
        
        Today's date is {current_date}.
        - Always calculate dates relative to today
        - Use tools for all reservations and date calculations
        - Confirm dates with users using MM/DD/YYYY format
        
        For general inquiries:
        - Answer questions about amenities, services, policies, and local attractions
        - If you don't have specific information, politely say so and offer to connect the guest with hotel staff
        
        Always be polite, professional, and helpful. If you need more information to assist a guest, ask clarifying questions."""),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ])
    return prompt

In [29]:
def agent_creation(tool_llm, tools, prompt):
    agent = create_tool_calling_agent(tool_llm, tools, prompt)
    # i've intentionally set verbose=True so that i can get to know when tool calling has happened.
    # set it to False if you just want the final response from LLM.
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    return agent, agent_executor

In [30]:
def chat():
    while True:
        user_input = input("User: ")
    
        if user_input=="bye":
            print("Alright, bye!")
            break
        
        # here i'm invoking agent along with all the previous conversations 
        # which has happened between the llm and user to make it more context aware.
        response = agent_executor.invoke({
            "input": user_input,
            "chat_history": chat_history
        })
        
        # print(f"chat history: {chat_history}") # to verify that chat history is being stored or not.
        
        if not isinstance(response, dict) or "interrupted" not in response:
            # updating the chat history if the tool calling was not interrupted
            chat_history.append(HumanMessage(content=user_input))
            chat_history.append(AIMessage(content=response["output"]))
        
        print(f"Assistant: {response["output"]}")

In [31]:
tool_llm, tools = integration_with_tool()
chat_history = []
prompt = system_prompt()
agent, agent_executor = agent_creation(tool_llm, tools, prompt)
print("Type 'bye' to end the conversation.")
chat()

Type 'bye' to end the conversation.


User:  hi




> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.
Assistant: Hello! How can I assist you today?


User:  Tell me more about the lalit




> Entering new AgentExecutor chain...
The Lalit Hotel offers a range of amenities and services, including comfortable rooms, a fitness center, a spa, and several dining options. We also have a concierge service that can help with local attractions and activities. Would you like to know more about any specific aspect?

> Finished chain.
Assistant: The Lalit Hotel offers a range of amenities and services, including comfortable rooms, a fitness center, a spa, and several dining options. We also have a concierge service that can help with local attractions and activities. Would you like to know more about any specific aspect?


...



> Entering new AgentExecutor chain...
What type of room would you like to book?

> Finished chain.
Assistant: What type of room would you like to book?


User:  deluxe




> Entering new AgentExecutor chain...

Invoking: `book_room` with `{'check_in_date': '03/09', 'check_out_date': '03/15', 'room_type': 'deluxe'}`


Your deluxe room with room number 37 has been successfully booked for March 9th to March 15th. Is there anything else I can assist you with?

> Finished chain.
Assistant: Your deluxe room with room number 37 has been successfully booked for March 9th to March 15th. Is there anything else I can assist you with?


User:  bye


Alright, bye!
